<span style="font-family: Arial; font-size: 14pt;"><b>Intermediate Training and Fine-tuning of BERT on Geneva Bible</b></span><br><br>
<span style="font-family: Arial; font-size: 12pt;">Author: Lucas Ma</span>

The first chunk of code loads the pretrained model in the repo. Note: the directory of the trained model is purposefully untracked by git. Please do not attempt to track it by "git add XXX" as it prevents you from pushing successfully to the remote main branch i.e. our GitHub repo.

In [77]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the fine-tuned MacBERTh model and tokenizer
model_name = './fine-tuned-MacBERTh'  # Path to your fine-tuned model
# model_name = "emanjavacas/MacBERTh"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Now get embedding.

In [78]:
def get_word_embedding(word, tokenizer, model):
    # Tokenize the input word
    inputs = tokenizer(word, return_tensors='pt')
    
    # Get the hidden states from the model
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # Extract the last hidden state for the input token
    hidden_states = outputs.hidden_states[-1]  # Shape: (batch_size, sequence_length, hidden_size)
    
    # Get the embedding for the first token (the input word)
    word_embedding = hidden_states[0, 1, :]  # Assuming the word is at position 1
    return word_embedding

# Example words
word1 = "god"
word2 = "love"

# Get embeddings for the words
embedding1 = get_word_embedding(word1, tokenizer, model)
embedding2 = get_word_embedding(word2, tokenizer, model)


Now compute cosine similarity.

In [79]:
from torch.nn.functional import cosine_similarity

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0)).item()

# Compute the cosine similarity between the embeddings
cosine_sim = calculate_cosine_similarity(embedding1, embedding2)
print(f"Cosine Similarity between '{word1}' and '{word2}': {cosine_sim}")


Cosine Similarity between 'god' and 'love': 0.950132429599762
